Julia is much faster than languages such as R, Python, and Matlab. Internally it is actually similar to compilied languages like C and Fortran. The way this works is that instead of interpreting your functions line by line like typical dynamic languages, Julia will check the types of the function arguments and compile it to machine code (as with C).

Internally, Julia transforms your Julia code into LLVM code. LLVM is an intermediate language. Then, the LLVM compiler transforms the LLVM code into machine code.

For reference, you can do the same thing with C. The Clang C compiler transforms C code into LLVM code, and then LLVM transforms that into machine code. Clang has similar performance to other C compilers. Given this, it is less surprising that Julia code is fast.

We can look at how this happens along the transformation pipeline.

In [12]:
function f(x)
    a = x*2 + 3
end
f(2.)

7.0

First, we check what methods Julia is going to call once the input type (Float64) is checked and multiple dispatch is performed.

In [13]:
@code_typed f(2.)

CodeInfo(
│╻╷ *2 1 ─ %1 = (Base.mul_float)(x, 2.0)::Float64
││╻  +  │   %2 = (Base.add_float)(%1, 3.0)::Float64
│    └──      return %2
) => Float64

Next, we can see the LLVM code

In [14]:
@code_llvm f(2.)


; Function f
; Location: In[12]:2
define double @julia_f_37230(double) {
top:
; Function *; {
; Location: promotion.jl:314
; Function *; {
; Location: float.jl:399
  %1 = fmul double %0, 2.000000e+00
;}}
; Function +; {
; Location: promotion.jl:313
; Function +; {
; Location: float.jl:395
  %2 = fadd double %1, 3.000000e+00
;}}
  ret double %2
}


In [15]:
Finally, we can look at the machine code (assembly language), which is mostly unreadable by humans.

LoadError: syntax: extra token "can" after end of expression

In [16]:
@code_native f(2.)

	.text
; Function f {
; Location: In[12]:2
; Function *; {
; Location: promotion.jl:314
; Function *; {
; Location: In[12]:2
	vaddsd	%xmm0, %xmm0, %xmm0
	movabsq	$140233780371888, %rax  # imm = 0x7F8AB8AA21B0
;}}
; Function +; {
; Location: promotion.jl:313
; Function +; {
; Location: float.jl:395
	vaddsd	(%rax), %xmm0, %xmm0
;}}
	retq
	nopw	%cs:(%rax,%rax)
;}
